In [282]:
import numpy as np
from scipy.special import expit
from scipy.optimize import fmin_bfgs
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [283]:
df = pd.read_csv('star_classification.csv')

In [284]:
df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [285]:
# Get rid of all rows which have class as GALAXY
df = df[df['class'] != 'GALAXY']

In [286]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis=1), df['class'], test_size=0.2, random_state=42)

In [287]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# encode the labels:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [903]:
class LogisticRegressionBase:
    def __init__(self, eta=0.1, iterations=20, C1=0.0001, C2 = 0.0001):
        self.eta = eta
        self.iters = iterations
        self.C1 = C1
        self.C2 = C2
    def __str__(self):
        if(hasattr(self, 'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
    @staticmethod
    def _add_bias(X):   
        return np.hstack((np.ones((X.shape[0], 1)), X))

    @staticmethod
    def _sigmoid(theta):
        return expit(theta)
    def _get_gradient(self, X, y):
        ydiff = y - self.predict_proba(X, add_bias = False).ravel()
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(self.w_.shape)
        return gradient
    def _get_gradient_L2(self, X, y):
        gradient = self._get_gradient(X, y)
        gradient[1:] += -2 * self.w_[1:] * self.C2
        return gradient
    def _get_gradient_L1(self, X, y):
        gradient = self._get_gradient(X, y)
        l1_der = self.w_[1:] / np.abs(self.w_[1:])
        l1_der[self.w_[1:] == 0] = 0
        gradient[1:] +=  -1 * l1_der * self.C1
        return gradient
    def _get_gradient_elastic(self, X, y):
        gradient = self._get_gradient(X, y)
        l1_der = self.w_[1:] / np.abs(self.w_[1:])
        l1_der[self.w_[1:] == 0] = 0
        gradient[1:] +=  -1 * l1_der * self.C1
        gradient[1:] += -2 * self.w_[1:] * self.C2
        return gradient
    def predict_proba(self, X, add_bias=True):
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_)

    def predict(self, X):
        return self.predict_proba(X) > 0.5
    def fit(self, X, y, regularization=None):
        Xb = self._add_bias(X)
        num_samples, num_features = Xb.shape
        self.w_ = np.random.uniform(-1, 1, (num_features, 1))
        for i in range(self.iters):
            if(regularization == 'L1'):
                grad = self._get_gradient_L1(Xb, y)
            elif(regularization == 'L2'):
                grad = self._get_gradient_L2(Xb, y)
            elif(regularization == 'elastic'):
                grad = self._get_gradient_elastic(Xb, y)
            else:
                grad = self._get_gradient(Xb, y)
            self.w_ += grad*self.eta

In [904]:
model = LogisticRegressionBase(eta=0.1, iterations=1000)
model.fit(X_train, y_train, regularization='elastic')
y_pred = model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))

Accuracy:  0.9707804216496116


In [259]:
class LogisticRegressionSGD(LogisticRegressionBase):
    def _get_gradient(self, X, y):
        sample = int(np.random.rand()  * len(y))
        ydiff = y[sample] - self.predict_proba(X[sample],add_bias=False)
        gradient = X[sample] * ydiff[:, np.newaxis]
        gradient = gradient.reshape(self.w_.shape)

        return gradient

In [874]:
class LogisticRegressionNewtons(LogisticRegressionBase):
    def _get_gradient(self, X, y):
        g = self.predict_proba(X, add_bias=False).ravel()
        hessian = X.T @ np.diag(g * (1-g)) @ X
        ydiff = y - g
        gradient = np.sum(X * ydiff[:, np.newaxis], axis=0)
        gradient = gradient.reshape(self.w_.shape)
        return np.linalg.pinv(hessian) @ gradient
    def _get_gradient_L1(self, X, y):
        g = self.predict_proba(X, add_bias=False).ravel()
        hessian = X.T @ np.diag(g * (1-g)) @ X # the second derivative of abs(x) evaluates to 0 so our hessian will simply be the one for the ordinary log likelihood
        ydiff = y - g
        gradient = np.sum(X @ ydiff[:, np.newaxis], axis=0)
        gradient = gradient.reshape(self.w_.shape)
        l1_der = self.w_[1:] / np.abs(self.w_[1:])
        l1_der[self.w_[1:] == 0] = 0
        gradient[1:] += -1 * l1_der[1:] * self.C1
        return np.linalg.pinv(hessian) @ gradient
    def _get_gradient_L2(self, X, y):
        g = self.predict_proba(X, add_bias=False).ravel()
        hessian = X.T @ np.diag(g * (1-g)) @ X - 2 * self.C2
        ydiff = y - g
        gradient = np.sum(X * ydiff[:, np.newaxis], axis=0)
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C2
        return np.linalg.pinv(hessian) @ gradient
    def _get_gradient_elastic(self, X, y):
        g = self.predict_proba(X, add_bias=False).ravel()
        hessian = X.T @ np.diag(g * (1-g)) @ X - 2 * self.C2
        ydiff = y - g
        gradient = np.sum(X @ ydiff[:, np.newaxis], axis=0)
        gradient = gradient.reshape(self.w_)
        l1_der = self.w_ / np.abs(self.w_)
        gradient[1:] += -1 * l1_der[1:] * self.C1
        gradient[1:] += -2 * self.w_[1:] * self.C2
        return np.linalg.pinv(hessian) @ gradient


In [875]:
model = LogisticRegressionNewtons(eta=0.1, iterations=10)
model.fit(X_train, y_train, regularization='L2')
y_pred = model.predict(X_test)
print(model)
print('Accuracy: ', accuracy_score(y_test, y_pred))

Binary Logistic Regression Object with coefficients:
[[-9.40343349e-02]
 [ 4.98958754e-01]
 [ 1.52642719e-01]
 [-2.54707938e-01]
 [ 1.89886327e+01]
 [-3.48716796e+01]
 [ 5.84916770e-01]
 [ 5.30809029e-01]
 [-1.79918575e+01]
 [-1.29219938e-01]
 [ 1.36337110e+02]
 [-3.15769960e-01]
 [-1.10324157e-01]
 [-3.93870990e-01]
 [-1.80952813e+00]
 [-1.58179032e-02]
 [ 2.75764412e-01]
 [-1.42204671e-01]]
Accuracy:  0.9351497965725558


In [1208]:
from numpy import ma
from scipy.optimize import fmin_bfgs
class BFGSLogisticRegression(LogisticRegressionBase):
    @staticmethod
    def objective_function(w,X,y,C1,C2):
        g = expit(X @ w)
        return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C2*sum(w**2) + C1*sum(np.abs(w))

    @staticmethod
    def objective_gradient(w,X,y,C1, C2):
        g = expit(X @ w)
        ydiff = y-g
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C2
        l1_der = w[1:] / np.abs(w[1:])
        l1_der[w[1:] == 0] = 0
        gradient[1:] +=  -1 * l1_der * C1

        return -gradient


    def fit(self, X, y, regularization=None):
        Xb = self._add_bias(X)
        num_samples, num_features = Xb.shape
        #modifying regularization params so we can use one objective function and gradient in all cases
        if(regularization == 'L1'):
            self.C2 = 0
        elif(regularization == 'L2'):
            self.C1 = 0
        elif(regularization == 'elastic'):
            pass
        else:
            self.C1 = 0
            self.C2 = 0

        self.w_ = fmin_bfgs(self.objective_function,
                        np.zeros((num_features,1)),
                        fprime=self.objective_gradient,
                        args=(Xb,y, self.C1, self.C2),
                        gtol=1e-03,
                        maxiter=self.iters,
                        disp=False)

        self.w_ = self.w_.reshape((num_features,1))


In [1209]:
bfgslr = BFGSLogisticRegression(iterations=200, C2=0.001, C1=0.001)
bfgslr.fit(X_train, y_train, regularization='elastic')
print(bfgslr)
print(accuracy_score(y_test, bfgslr.predict(X_test)))

Binary Logistic Regression Object with coefficients:
[[-1.63015499e+00]
 [-1.83140808e-01]
 [-1.00817179e-01]
 [-3.92314450e-02]
 [ 1.57771136e-02]
 [-3.85320072e-03]
 [ 1.19140957e-01]
 [-7.09587864e-01]
 [-3.65967384e-02]
 [-1.83144369e-01]
 [ 0.00000000e+00]
 [ 4.08749496e-03]
 [-1.47740638e-02]
 [-9.99228090e-03]
 [-6.03358052e+00]
 [-9.99305125e-03]
 [ 3.47034543e-01]
 [-1.52369997e-02]]
0.9749722598939712


In [1272]:
class BFGSFromScratchLogisticRegression(LogisticRegressionBase):


    def fit(self, X, y, regularization=None):
        if(regularization == 'L1'):
            self.C2 = 0
        elif(regularization == 'L2'):
            self.C1 = 0
        elif(regularization == 'elastic'):
            pass
        else:
            self.C1 = 0
            self.C2 = 0
        Xb = self._add_bias(X)
        num_samples, num_features = Xb.shape
        self.w_ = np.zeros((num_features, 1))
        self.inv_hessian = np.identity(num_features)
        self.last_grad = np.zeros((num_features,1))
        g = self.predict_proba(Xb, add_bias=False).ravel()
        ydiff = y-g
        self.last_grad = np.sum(Xb * ydiff[:, np.newaxis], axis=0)
        self.last_grad = self.last_grad.reshape((num_features, 1))
        self.last_grad[1:] += -2 * self.w_[1:] * self.C2
        l1_der = self.w_[1:] / np.abs(self.w_[1:])
        l1_der[self.w_[1:] == 0] = 0
        self.last_grad[1:] +=  -1 * l1_der * self.C1
        self.last_grad = -self.last_grad

        for i in range(self.iters):
            pk = -np.dot(self.inv_hessian, self.last_grad)
            pk = pk.reshape((num_features, 1))
            sk = self.eta * pk
            self.w_ += sk
            g = self.predict_proba(Xb, add_bias=False).ravel()
            ydiff = y-g
            curr_grad = np.sum(Xb * ydiff[:, np.newaxis], axis=0)
            curr_grad = curr_grad.reshape((num_features, 1))
            curr_grad[1:] += -2 * self.w_[1:] * self.C2
            l1_der = self.w_[1:] / np.abs(self.w_[1:])
            l1_der[self.w_[1:] == 0] = 0
            curr_grad[1:] +=  -1 * l1_der * self.C1
            curr_grad = -curr_grad
            vk = curr_grad - self.last_grad
            print(np.linalg.norm(vk))
            if(np.linalg.norm(vk) < 3):
                print("stopping condition at iteration", i+1)
                break
            inv_hessian_num_1_1 = (sk.T @ vk) + self.inv_hessian
            inv_hessian_num_1_2 = sk @ sk.T
            inv_hessian_num_1 = inv_hessian_num_1_1 @ inv_hessian_num_1_2
            inv_hessian_dom_1 = (sk.T @ vk) ** 2
            inv_hessian_1 = inv_hessian_num_1 / inv_hessian_dom_1
            inv_hessian_num_2 = (self.inv_hessian @ vk @ sk.T) + (sk @ vk.T @ self.inv_hessian)
            inv_hessian_dom_2 =  sk.T @ vk
            inv_hessian_2 = inv_hessian_num_2 / inv_hessian_dom_2
            self.inv_hessian += inv_hessian_1 - inv_hessian_2
            self.last_grad = curr_grad




In [1273]:
test_model = BFGSFromScratchLogisticRegression(iterations=1000, eta=0.001)
test_model.fit(X_train, y_train, regularization='elastic')
print(test_model)
print(accuracy_score(y_test, test_model.predict(X_test)))

28863.186932000404
2414.29579027471
811.6821660112919
1336.5351110118002
450.89748118919397
1190.5326834092527
347.26472173339766
1184.3102204611541
471.83392975034707
903.893069628268
400.0865434170528
1136.35245348917
512.998463797033
1219.437945531901
687.3096406380832
1287.6839224712464
763.1834669021915
1346.8504716518469
836.7890622318582
1239.6941917184245
842.6916270493907
1353.1831118886296
848.7408711093912
1261.3923488989192
835.6889331377193
1220.2572006825228
807.7222404045743
1109.2149216635346
727.2328527315137
925.3638952517579
644.0131941041502
739.7264988766927
424.2875871998262
600.8346418435495
335.5743204761952
491.3324780517168
281.4116248468743
417.3871226819175
254.05911973239694
370.94053625399073
229.75768669356265
335.0039271683607
223.1751567039706
315.2322864463399
209.25131058809157
293.1159306137717
201.41435460544068
274.03504736674034
197.498194310172
255.6030800553616
187.03481870182594
233.16679349422586
173.63678733028638
210.9409111234655
159.380088

In [1217]:
class LogisticRegression:
    def __init__(self, eta=.1, iters=10, C1=.001, C2=.0001, solver="default", regularization=None):
        self.eta = eta
        self.iters = iters
        self.C1 = C1
        self.C2 = C2
        self.solver = solver
        self.classifiers = []
        self.regularization = regularization

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    def fit(self, X, y):
        # Get number of unique values of y
        unique_classes = np.unique(y)
        unique_classes.sort()
        for target in unique_classes:
            # Transform the data into binary classification, the taget class vs the rest
            y_binary = np.where(y == target, 1, 0)
            if self.solver == "default":
                model = LogisticRegressionBase(iterations=self.iters, eta=self.eta, C1=self.C1, C2=self.C2)
            elif self.solver == "sgd":
                model = LogisticRegressionSGD(iterations=self.iters, eta=self.eta, C1=self.C1, C2=self.C2)
            elif self.solver == "newton":
                model = LogisticRegressionNewtons(iterations=self.iters, eta=self.eta, C1=self.C1, C2=self.C2)
            elif self.solver == 'bfgs':
                model = BFGSLogisticRegression(iterations=self.iters, eta=self.eta, C1=self.C1, C2=self.C2)
            elif self.solver == 'bfgs_scratch':
                model = BFGSFromScratchLogisticRegression(iterations=self.iters, eta=self.eta, C1=self.C1, C2=self.C2)
            model.fit(X, y_binary, regularization=self.regularization)
            self.classifiers.append(model)
        self.w_ = np.hstack([x.w_ for x in self.classifiers]).T
    def predict_proba(self, X):
        probs = []
        for model in self.classifiers:
            probs.append(model.predict_proba(X).reshape(len(X), 1))
        return np.hstack(probs)

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)


In [879]:
model = LogisticRegression(iters=150000, eta=1, solver="sgd", regularization="l2")
model.fit(X_train, y_train)
%time
y_hat = model.predict(X_test)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 3.1 µs


In [880]:

# Calculate accuracy of y_hat vs y_test with sklearn
print('Accuracy: ', accuracy_score(y_test, y_hat))

Accuracy:  0.9939588213537172


In [269]:
model = LogisticRegression(iters=10, eta=.1, solver="newton", regularization="l2")
model.fit(X_train, y_train)
%time
y_hat_new = model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_hat_new))

CPU times: user 3 µs, sys: 5 µs, total: 8 µs
Wall time: 4.77 µs
Accuracy:  0.814


In [245]:
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
SKmodel = SKLogisticRegression(solver='lbfgs', penalty='l2', max_iter=1000)
SKmodel.fit(X_train, y_train)
%time
print(accuracy_score(y_true=y_test, y_pred=SKmodel.predict(X_test)))

CPU times: user 3 µs, sys: 4 µs, total: 7 µs
Wall time: 3.1 µs
0.95485
